# TECHNICAL TEST : EXPLORATORY DATA ANALYSIS

## Context

A big prospect has recently contacted us because he has data quality issues. 

The prospect told us that he receives “FEDAS codes” from its suppliers. However those FEDAS codes are often incorrect, and a team of 5 people is currently mobilized full time to check these. Let’s automate this for them!

The prospect sent us a train dataset, in which we can find the original FEDAS codes (***incorrect_fedas_code***) and the manually corrected FEDAS codes (***correct_fedas_code***)

Our goal is to build an algorithm able to correct the FEDAS codes by predicting the column **correct_fedas_code**.

In [1]:
from importlib import reload
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import utils


The cleaning functions that we'll test in this notebook will be implemented in a separate module `utils.py`. 

## Data exploration

In [2]:
raw_train = pd.read_csv('data_technical_test/train_technical_test.csv', 
    na_values="",
    dtype={
        "incorrect_fedas_code": object, 
        "correct_fedas_code": object, 
    },
    parse_dates=["avalability_start_date", "avalability_end_date"])

In [3]:
train = raw_train.copy(deep=True)

In [4]:
train["incorrect_fedas_code"] = train["incorrect_fedas_code"].fillna("")

In [5]:
train.shape

(39322, 31)

In [6]:
train.columns

Index(['brand', 'model_code', 'model_label', 'commercial_label',
       'incorrect_fedas_code', 'article_main_category', 'article_type',
       'article_detail', 'comment', 'avalability_start_date',
       'avalability_end_date', 'length', 'width', 'height', 'color_code',
       'color_label', 'inaccurate_gender', 'country_of_origin',
       'country_of_manufacture', 'embakment_harbor', 'shipping_date',
       'eco_participation', 'eco_furniture', 'multiple_of_order',
       'minimum_multiple_of_order', 'net_weight', 'raw_weight', 'volume',
       'size', 'correct_fedas_code', 'accurate_gender'],
      dtype='object')

In [7]:
train

,brand,model_code,model_label,commercial_label,incorrect_fedas_code,article_main_category,article_type,article_detail,comment,avalability_start_date,...,eco_participation,eco_furniture,multiple_of_order,minimum_multiple_of_order,net_weight,raw_weight,volume,size,correct_fedas_code,accurate_gender
0,brand_293,S42783,FLEXAGON ENERGY TR 3.0 MT,NaN,378011,TRAINING,HOMME,09-SHOES (LOW),NaN,2020-12-01,...,0.0,0.0,1,0,0.00,0.00,0.0,38.5,378101,HO
1,brand_3,R1252,TADEN PLUS FUR,NaN,,GARDEN,RUBBER BOOTS,BOOTS,NaN,2020-01-01,...,0.0,0.0,1,1,0.00,0.00,0.0,36,364308,FE
2,brand_265,OXS917808,POCHETTE PORTE TRAVERS PE,NaN,175897,SAC,HOMME,N1FARROW,MATERIEL RANDONNEE,2021-01-01,...,0.0,0.0,0,0,0.00,0.00,0.0,U,175890,UA
3,brand_1,GM5253,CLUB KNOT TANK,NaN,224122,RACKET SPORTS,FEMME,21-TANK,NaN,2020-12-01,...,0.0,0.0,1,0,0.00,0.00,0.0,2XS,224118,FE
4,brand_12,MS338,BONITA DK PNK/BLCK M,NaN,,NaN,NaN,NaN,SNO,2020-01-01,...,0.0,0.0,6,0,0.00,0.00,0.0,M,115944,UA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39317,brand_152,57502669I,ICEPEAK BAUTZEN,ICEPEAK BAUTZEN,,OUTDOOR ADVENTURE,SHORT,NaN,NaN,2020-06-01,...,0.0,0.0,1,1,0.13,0.15,0.0,46,264701,HO
39318,brand_329,CSECURUN12,SEMELLES RUN CUSTOM,NaN,100981,RUNNING,UNISEX,SEMELLE,NaN,2019-02-01,...,0.0,0.0,0,0,0.00,0.00,0.0,L,146981,UA
39319,brand_17,2032B756,KATAKANA GRAPHIC TEE,NaN,278135,TRAINING,FEMME,KATAKANA GRAPHIC T,NaN,2021-01-15,...,0.0,0.0,1,0,0.00,0.00,0.0,L,200125,FE
39320,brand_1,FM9969,ESSENTIAL TEE,NaN,275124,SPORTSTYLE,HOMME,27-T-SHIRT (SHORT SLEEVE),NaN,2020-05-01,...,0.0,0.0,1,0,0.00,0.00,0.0,L,200124,HO


In [8]:
train.describe(include='all', datetime_is_numeric=True)

,brand,model_code,model_label,commercial_label,incorrect_fedas_code,article_main_category,article_type,article_detail,comment,avalability_start_date,...,eco_participation,eco_furniture,multiple_of_order,minimum_multiple_of_order,net_weight,raw_weight,volume,size,correct_fedas_code,accurate_gender
count,39322,39322,39322,6238,39322,38571,38402,29622,1552,24908,...,39322.000000,39322.000000,39322.000000,39322.000000,39322.000000,39322.000000,39322.000000,39322,39322,39322
unique,329,38715,29558,4772,2188,710,1221,4076,134,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,812,1468,11
top,brand_1,813271-40,MAN JEANS,TBT_AP_MN TOP,,LOISIRS,HOMME,09-SHOES (LOW),VETEMENT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L,275124,HO
freq,6089,4,86,20,10854,2855,3906,1436,413,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7448,642,14775
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-15 17:42:08.151597824,...,0.005552,0.000080,2.780937,10.425055,5.021437,2.270721,1.545066,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-01-01 00:00:00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-15 00:00:00,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-01 00:00:00,...,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-12-01 00:00:00,...,0.000000,0.000000,1.000000,1.000000,0.130000,0.000000,0.000000,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-25 00:00:00,...,3.150000,3.150000,324.000000,5000.000000,12500.000000,14000.000000,1010.000000,NaN,NaN,NaN


## Business information

A rapid Google search shows that the FEDAS code is built from the following information ([source](https://www.sgidho.com/FR/SiteAssets/SitePages/Introduction/Introduction%20V%204.0.pdf)):

- Digit 1: Product type (Hardware, Footwear, Textile, Service, Rental)
- Digit 2 and 3: Activity code (type of sports)
- Digit 4 and 5: Product Maingroup of the activity
- Digit 6: Product Subgroup


But things are not that simple and the type, activity, etc, are not clearly indicated in such a way that a few expert rules may be sufficient. Moreover there may be errors.

So let's split the the fedas code into 4 groups of digits (1, 23, 45, 6)  and observe the correlations again.

We'll split the incorrect fedas code as well. This will ease analysis and also be part of feature engineering. Maybe parts of the incorrect fedas are correct and may be used to predict the correct fedas?


In [9]:
correct_fedas_columns = [f"correct_fedas_{i}" for i in range(1, 5)]
incorrect_fedas_columns = [f"incorrect_fedas_{i}" for i in range(1, 5)]
train[correct_fedas_columns] = train.correct_fedas_code.apply(utils.split_fedas_code).apply(pd.Series)    
train[incorrect_fedas_columns] = train.incorrect_fedas_code.apply(utils.split_fedas_code).apply(pd.Series)

In [10]:
train.drop(columns=["correct_fedas_code"], inplace=True)

In [11]:
# profile = ProfileReport(train, title="Pandas Profiling Report", explorative=True)
# profile.to_file("train_report.html")

The only noticeable linear correlation between correct fedas codes and other elements is between `correct_code_1` and `incorrect_code_1`. Which is normal since the category section is the one with smallest number of classes (4). 

In [12]:
np.corrcoef(train.correct_fedas_1, train.incorrect_fedas_1)

array([[1.        , 0.39386688],
       [0.39386688, 1.        ]])

But we'd like to be able to predict the correct_fedas_code only from other features. So we'll drop the `incorrect_fedas_code`.

## Feature engineering

### Drop columns

In [38]:
train = raw_train.copy(deep=True)

In [39]:
target = pd.DataFrame(train.correct_fedas_code)

In [40]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39322 entries, 0 to 39321
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   brand                      39322 non-null  object        
 1   model_code                 39322 non-null  object        
 2   model_label                39322 non-null  object        
 3   commercial_label           6238 non-null   object        
 4   incorrect_fedas_code       28468 non-null  object        
 5   article_main_category      38571 non-null  object        
 6   article_type               38402 non-null  object        
 7   article_detail             29622 non-null  object        
 8   comment                    1552 non-null   object        
 9   avalability_start_date     24908 non-null  datetime64[ns]
 10  avalability_end_date       22004 non-null  datetime64[ns]
 11  length                     39322 non-null  float64       
 12  widt

Among the features we select those that seem most relevant (these are the elements that we assume the correcting team use to correct the fedas code; this should be asked to the client):

In [17]:
train = train[['brand',
       'article_main_category', 'article_type', 'article_detail', 
       'comment', 'size', 'accurate_gender']]

Note: the model_label feature should deserve to be kept, but my computer is not powerful enough to handle the vectorization (16Mo RAM).

In [18]:
target[correct_fedas_columns] = target.correct_fedas_code.apply(utils.split_fedas_code).apply(pd.Series)

In [19]:
target.drop(columns=["correct_fedas_code"], inplace=True)

In [20]:
target

,correct_fedas_1,correct_fedas_2,correct_fedas_3,correct_fedas_4
0,3,78,10,1
1,3,64,30,8
2,1,75,89,0
3,2,24,11,8
4,1,15,94,4
...,...,...,...,...
39317,2,64,70,1
39318,1,46,98,1
39319,2,0,12,5
39320,2,0,12,4


In [21]:
for col in correct_fedas_columns:
    print(f'For {col}, there are {target[col].nunique()} unique values.')

For correct_fedas_1, there are 4 unique values.
For correct_fedas_2, there are 44 unique values.
For correct_fedas_3, there are 99 unique values.
For correct_fedas_4, there are 10 unique values.


Since these codes are categorical, there is no need for normalization / standardization.

We'll consider that the different parts of fedas code are not correlated, so we'll train a model for each part.

Now we need to encode the categorical features.

In [22]:
train.describe()

,brand,article_main_category,article_type,article_detail,comment,size,accurate_gender
count,39322,38571,38402,29622,1552,39322,39322
unique,329,710,1221,4076,134,812,11
top,brand_1,LOISIRS,HOMME,09-SHOES (LOW),VETEMENT,L,HO
freq,6089,2855,3906,1436,413,7448,14775


In [23]:
train.article_main_category.value_counts()

LOISIRS                       2855
FOOTBALL                      2518
TRAINING                      2498
SPORTSTYLE                    2310
LOISIR                        1770
                              ... 
BAGS WAIST BAG                   1
GOODMORNING                      1
TRICOLORE                        1
TERRE                            1
CLAVAS (BALA, HELMA, THIN)       1
Name: article_main_category, Length: 710, dtype: int64

### Normalization

Let's normalize the strings columns:


In [24]:
reload(utils);

In [25]:
for col in train.columns:
    if col in ['brand', 'size']:
        train[col] = train[col].apply(lambda text: utils.normalize(text, keep_digits=True))
    else:
        train[col] = train[col].apply(lambda text: utils.normalize(text))

In [26]:
train

,brand,article_main_category,article_type,article_detail,comment,size,accurate_gender
0,brand 293,training,homme,shoes low,,38 5,ho
1,brand 3,garden,rubber boots,boots,,36,fe
2,brand 265,sac,homme,n farrow,materiel randonnee,u,ua
3,brand 1,racket sports,femme,tank,,2xs,fe
4,brand 12,,,,sno,m,ua
...,...,...,...,...,...,...,...
39317,brand 152,outdoor adventure,short,,,46,ho
39318,brand 329,running,unisex,semelle,,l,ua
39319,brand 17,training,femme,katakana graphic t,,l,fe
39320,brand 1,sportstyle,homme,t shirt short sleeve,,l,ho


In [27]:
train.describe()

,brand,article_main_category,article_type,article_detail,comment,size,accurate_gender
count,39322,39322,39322,39322,39322,39322,39322
unique,329,699,1193,3843,114,754,11
top,brand 1,loisirs,homme,,,l,ho
freq,6089,2855,3906,10158,37773,7643,14775


The columns are not always in the expected order. For instance in line 2, the comment "matériel de randonnée" should be in the column `category`. Moreover, we are trying to reproduce the behavior of the correcting team, so we'll try to predict the correct fedas code ***from a string composed of all the strings on the row***.

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

vectorizer = TfidfVectorizer()

In [29]:
inputs = pd.DataFrame(index = train.index, columns = ['text'])

In [30]:
inputs['text'] = train.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [31]:
inputs

,text
0,brand 293 training homme shoes low 38 5 ho
1,brand 3 garden rubber boots boots 36 fe
2,brand 265 sac homme n farrow materiel randonne...
3,brand 1 racket sports femme tank 2xs fe
4,brand 12 sno m ua
...,...
39317,brand 152 outdoor adventure short 46 ho
39318,brand 329 running unisex semelle l ua
39319,brand 17 training femme katakana graphic t l fe
39320,brand 1 sportstyle homme t shirt short sleeve ...


We'll use a **Random Forest Classifier** to predict each part of the fedas code, since it seems appropriate: we suspect that there is correlation between each part of fedas code and the previous part. And since we want to output 4 values, we'll use a **multi-output classifier**.

We implement a custom cross-validation.

In [32]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics  import accuracy_score

In [33]:
forest = RandomForestClassifier(n_estimators=100, random_state=42)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)

In [34]:
results = pd.DataFrame(columns=['fedas'] + correct_fedas_columns)

In [35]:
for random_state in tqdm(range(5)):

    # Split the data into training (90%) and test 10%) sets
    X_train, X_test, y_train, y_test = train_test_split(inputs, target, 
        test_size=0.1, random_state=random_state)
    
    # Fit embedding on the training data (to avoid info leak)
    X_train = vectorizer.fit_transform(X_train['text'])
    X_test = vectorizer.transform(X_test['text'])
    
    # Fit model on the training data
    multi_target_forest.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = multi_target_forest.predict(X_test)
    y_pred = pd.DataFrame(y_pred, columns=correct_fedas_columns)

    # Compute accuracy on each part
    for col in correct_fedas_columns:
        accuracy = accuracy_score(y_test[col], y_pred[col])
        results.loc[random_state, col] = accuracy

    # Compute accuracy on the whole fedas code
    merged_y_pred = y_pred.apply(lambda row: ''.join(row.values.astype(str)), axis=1)
    merged_y_test = y_test.apply(lambda row: ''.join(row.values.astype(str)), axis=1)
    results.loc[random_state, 'fedas'] = accuracy_score(merged_y_test, merged_y_pred)

  0%|          | 0/5 [00:00<?, ?it/s]

In [36]:
results.style.format("{:.1%}")

,fedas,correct_fedas_1,correct_fedas_2,correct_fedas_3,correct_fedas_4
0,68.4%,99.1%,91.1%,77.2%,79.5%
1,67.4%,99.3%,90.7%,76.8%,77.9%
2,68.6%,99.4%,90.9%,77.9%,78.9%
3,67.8%,99.2%,90.3%,77.3%,78.6%
4,67.4%,99.2%,90.4%,76.9%,78.2%
